# Conversational Memory with LCEL

Conversational memory is how chatbots can respond to our queries in a chat-like manner. It enables a coherent conversation, and without it, every query would be treated as an entirely independent input without considering past interactions.

The memory allows an _"agent"_ to remember previous interactions with the user. By default, agents are *stateless* — meaning each incoming query is processed independently of other interactions. The only thing that exists for a stateless agent is the current input, nothing else.

There are many applications where remembering previous interactions is very important, such as chatbots. Conversational memory allows us to do that.

In this notebook we'll explore conversational memory using modern LangChain Expression Language (LCEL) and the recommended `RunnableWithMessageHistory` class.

We'll start by importing all of the libraries that we'll be using in this example.

In [1]:
!pip install -qU \
  langchain==0.3.25 \
  langchain-community==0.3.25 \
  langchain-openai==0.3.22 \
  tiktoken==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.6 requires langchain-core>=1.0.0, but you have langchain-core 0.3.83 which is incompatible.


To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key if prompted, otherwise it will use the `OPENAI_API_KEY` environment variable.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
env_path = "/content/drive/MyDrive/.env"
# env_path = "C:\\Users\\vborg\\Git\\.env"
load_dotenv(env_path)

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4.1-mini'
)

Later we will make use of a `count_tokens` utility function. This will allow us to count the number of tokens we are using for each call. We define it as so:

In [7]:
from langchain.callbacks import get_openai_callback

def count_tokens(pipeline, query, config=None):
    with get_openai_callback() as cb:
        # Handle both dict and string inputs
        if isinstance(query, str):
            query = {"query": query}

        # Use provided config `or default
        if config is None:
            config = {"configurable": {"session_id": "default"}}

        result = pipeline.invoke(query, config=config)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

Now let's dive into **Conversational Memory** using LCEL.

## What is memory?

**Definition**: Memory is an agent's capacity of remembering previous interactions with the user (think chatbots)

The official definition of memory is the following:

> By default, Chains and Agents are stateless, meaning that they treat each incoming query independently. In some applications (chatbots being a GREAT example) it is highly important to remember previous interactions, both at a short term but also at a long term level. The concept of "Memory" exists to do exactly that.

As we will see, although this sounds really straightforward there are several different ways to implement this memory capability.

## Building Conversational Chains with LCEL

Before we delve into the different memory types, let's understand how to build conversational chains using LCEL. The key components are:

1. **Prompt Template** - Defines the conversation structure with placeholders for history and input
2. **LLM** - The language model that generates responses
3. **Output Parser** - Converts the LLM output to the desired format (optional)
4. **RunnableWithMessageHistory** - Manages conversation history

Let's create our base conversational chain:

In [8]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.schema.output_parser import StrOutputParser

# Define the prompt template
system_prompt = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."""

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

# Create the LCEL pipeline
output_parser = StrOutputParser()
pipeline = prompt_template | llm | output_parser

# Let's examine the prompt template
print(prompt_template.messages[0].prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.


## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.

### Memory Type #1: Buffer Memory - Store the Entire Chat History

`InMemoryChatMessageHistory` and `RunnableWithMessageHistory` are used as alternatives to `ConversationBufferMemory` as they are:
- More flexible and configurable.
- Integrate better with LCEL.

The simplest approach to using them is to simply store the entire chat in the conversation history. Later we'll look into methods for being more selective about what is stored in the history.

In [9]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create a simple chat history storage
chat_map = {}

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

Let's see this in action by having a conversation:

In [ ]:
# Create the conversational chain with message history
conversation_buf = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [10]:
# First message
result = conversation_buf.invoke(
    {"query": "Good morning AI!"},
    # Make sure to pass the session ID to ensure all memories are stored in the same session
    config={"configurable": {"session_id": "buffer_example"}}
)
print(result)

Good morning! How can I assist you today?


This call used some tokens, but we can't see that from the above.

If we'd like to count the number of tokens being used we just pass our conversation `RunnableWithMessageHistory` instance and the message we'd like to input to the `count_tokens` function we defined earlier:

In [11]:
# Continue the conversation with token counting
query1 = """
"My interest here is to explore the potential of integrating Large Language Models with external knowledge.

Also, remember this very specific code: PINECONE_RULEZ_01"
"""

result1 = count_tokens(
    conversation_buf,
    {"query": query1},
    # Make sure to pass the session ID to ensure all memories are stored in the same session
    config={"configurable": {"session_id": "buffer_example"}}
)
print(f"\nResponse: {result1}")

Spent a total of 180 tokens

Response: Got it! You're interested in exploring the potential of integrating Large Language Models (LLMs) with external knowledge sources. Also, I've noted the specific code: PINECONE_RULEZ_01. How would you like to proceed with this exploration? Are you looking for examples, technical details, or something else?


In [12]:
# Continue the conversation and, in the last iteration, ask for the specific code´
query2 = # INSERT YOUR CODE HERE
result2 = # INSERT YOUR CODE HERE
print(f"\nResponse: {result2}")

query3 = # INSERT YOUR CODE HERE
result3 = # INSERT YOUR CODE HERE
print(f"\nResponse: {result3}")

query4 = # INSERT YOUR CODE HERE
result4 = # INSERT YOUR CODE HERE
print(f"\nResponse: {result4}")

query5 = # INSERT YOUR CODE HERE
result5 = # INSERT YOUR CODE HERE
print(f"\nResponse: {result5}")

Spent a total of 726 tokens

Response: Great! Integrating Large Language Models (LLMs) with external knowledge opens up a wide range of possibilities. Here are some key approaches and use cases to consider:

1. **Vector Databases and Semantic Search**  
   - Use vector databases like Pinecone (which your code hints at!) to store embeddings of external documents or data.  
   - When a user query comes in, convert it into an embedding and perform a similarity search to retrieve relevant external knowledge.  
   - This allows the LLM to ground its responses in up-to-date or domain-specific information beyond its training data.

2. **Knowledge Graph Integration**  
   - Connect LLMs with structured knowledge graphs (e.g., Wikidata, custom enterprise graphs).  
   - The LLM can query the graph to retrieve precise facts or relationships, improving accuracy and enabling complex reasoning.

3. **API and Database Calls**  
   - Enable the LLM to trigger external API calls or database queries du

Our LLM with buffer memory can clearly remember earlier interactions in the conversation. Let's take a closer look at how the messages are being stored:

In [15]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Access the conversation history
history = chat_map["buffer_example"].messages
print("Conversation History:")
for i, msg in enumerate(history):
    if isinstance(msg, HumanMessage):
        role = "Human"
    elif isinstance(msg, SystemMessage):
        role = "System"
    elif isinstance(msg, AIMessage):
        role = "AI"
    else:
        role = "Unknown"
    print(f"{role}: {msg}")

Conversation History:
Human: content='Good morning AI!' additional_kwargs={} response_metadata={}
AI: content='Good morning! How can I assist you today?' additional_kwargs={} response_metadata={}
Human: content='\n"My interest here is to explore the potential of integrating Large Language Models with external knowledge.\n\nAlso, remember this very specific code: PINECONE_RULEZ_01"\n' additional_kwargs={} response_metadata={}
AI: content="Got it! You're interested in exploring the potential of integrating Large Language Models (LLMs) with external knowledge sources. Also, I've noted the specific code: PINECONE_RULEZ_01. How would you like to proceed with this exploration? Are you looking for examples, technical details, or something else?" additional_kwargs={} response_metadata={}
Human: content='I just want to analyze the different possibilities. What can you think of?' additional_kwargs={} response_metadata={}
AI: content='Great! Integrating Large Language Models (LLMs) with external 

Nice! So every piece of our conversation has been explicitly recorded and sent to the LLM in the prompt.

### Memory type #2: Summary - Store Summaries of Past Interactions

The problem with storing the entire chat history in agent memory is that, as the conversation progresses, the token count adds up. This is problematic because we might max out our LLM with a prompt that is too large.

The following is an LCEL compatible alternative to `ConversationSummaryMemory`. We keep a summary of our previous conversation snippets as our history. The summarization is performed by an LLM.

**Key feature:** _the conversation summary memory keeps the previous pieces of conversation in a summarized - and thus shortened - form, where the summarization is performed by an LLM._

In [16]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)

    def __init__(self, llm: ChatOpenAI):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history and update the summary."""
        self.messages.extend(messages)

        # Construct the summary prompt
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensure to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])

        # Format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=self.messages,
                messages=messages
            )
        )

        # Replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [17]:
from langchain_core.runnables import ConfigurableFieldSpec

# Create get_chat_history function for summary memory
summary_chat_map = {}

def get_summary_chat_history(session_id: str, llm: ChatOpenAI) -> ConversationSummaryMessageHistory:
    if session_id not in summary_chat_map:
        summary_chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    return summary_chat_map[session_id]

# Create conversation chain with summary memory
conversation_sum = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_summary_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
    ]
)

In [18]:
# Let's have the same conversation with summary memory
result = count_tokens(
    conversation_sum,
    {"query": "Good morning AI!"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 225 tokens

Response: Good morning! How can I assist you today?


In [ ]:
# INSERT YOUR CODE HERE
# Use the same sentences as before, and ask for the code at the end

In [23]:
# Let's examine the summary
print("Summary Memory Content:")
print(summary_chat_map["summary_example"].messages[0].content)

Summary Memory Content:
The user asked to be reminded of their aim and the specific code the AI was tasked with remembering. The AI clarified that the user's aim is to understand which data source types can be used to provide context to a Large Language Model (LLM). These data sources include text documents, databases, knowledge graphs, APIs, multimedia converted to text, user interaction logs, domain-specific repositories, and web-scraped data, all intended to enrich the LLM’s responses for greater accuracy, relevance, and timeliness. The AI also reiterated the specific code it was tasked with remembering: **PINECONE_RULEZ_01**. The AI offered further assistance on integration or related details if needed.


You might be wondering.. if the aggregate token count is greater in each call here than in the buffer example, why should we use this type of memory? Well, if we check out buffer we will realize that although we are using more tokens in each instance of our conversation, our final history is shorter. This will enable us to have many more interactions before we reach our prompt's max length, making our chatbot more robust to longer conversations.

We can count the number of tokens being used (without making a call to OpenAI) using the `tiktoken` tokenizer like so:

In [24]:
import tiktoken

# initialize tokenizer (gpt-4.1 models use the same encoding as gpt-4o)
tokenizer = tiktoken.encoding_for_model('gpt-4o')

# Get buffer memory content
buffer_messages = chat_map["buffer_example"].messages
buffer_content = "\n".join([msg.content for msg in buffer_messages])

# Get summary memory content
summary_content = summary_chat_map["summary_example"].messages[0].content

# show number of tokens for the memory used by each memory type
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(buffer_content))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(summary_content))}'
)

Buffer memory conversation length: 1274
Summary memory conversation length: 139


_Practical Note: the `gpt-4o-mini` model has a context window of 1M tokens, providing significantly more space for conversation history than older models._

### Memory type #3: Window Buffer Memory - Keep Latest Interactions

Another great option is window memory, where we keep only the last k interactions in our memory but intentionally drop the oldest ones - short-term memory if you'd like. Here the aggregate token count **and** the per-call token count will drop noticeably.

The following is an LCEL-compatible alternative to `ConversationBufferWindowMemory`.

**Key feature:** _the conversation buffer window memory keeps the latest pieces of the conversation in raw form_

In [25]:
class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        # Add logging to help with debugging
        print(f"Initializing BufferWindowMessageHistory with k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        # Add logging to help with debugging
        if len(self.messages) > self.k:
            print(f"Truncating history from {len(self.messages)} to {self.k} messages")
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [26]:
# Create get_chat_history function for window memory
window_chat_map = {}

def get_window_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    print(f"get_window_chat_history called with session_id={session_id} and k={k}")
    if session_id not in window_chat_map:
        window_chat_map[session_id] = BufferWindowMessageHistory(k=k)
    return window_chat_map[session_id]

# Create conversation chain with window memory
conversation_bufw = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_window_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

In [27]:
# Start a conversation with k=2 (only remembers last 2 exchanges = 4 messages)
result = count_tokens(
    conversation_bufw,
    {"query": "Good morning AI!"},
    config={"configurable": {"session_id": "window_example", "k": 4}}
)
print(f"\nResponse: {result}")

get_window_chat_history called with session_id=window_example and k=4
Initializing BufferWindowMessageHistory with k=4
Spent a total of 74 tokens

Response: Good morning! How can I assist you today?


In [ ]:
# INSERT YOUR CODE HERE
# Use the same sentences as before, and ask for the code at the end

As we can see, it effectively 'forgot' what we talked about in the first interaction. Let's see what it 'remembers':

In [32]:
# Check what's in memory
bufw_history = window_chat_map["window_example"].messages
print("Buffer Window Memory (last 4 messages):")
for msg in bufw_history:
    role = "Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"\n{role}: {msg.content}")  # Show first 100 chars

Buffer Window Memory (last 4 messages):

Human: Which data source types could be used to give context to the model?

AI: Great question! To give context to a Large Language Model (LLM) by integrating external knowledge, you can use a variety of data source types depending on your application and goals. Here are some common and effective data source types that can provide rich context:

1. **Textual Documents:**  
   - Articles, books, research papers, manuals, FAQs, and reports.  
   - These can be stored in databases or document stores and retrieved based on relevance.  
   - Useful for knowledge bases, customer support, academic research, etc.

2. **Structured Databases:**  
   - Relational databases (SQL), NoSQL databases, or data warehouses containing structured records.  
   - Examples: customer data, product catalogs, transaction logs.  
   - Enables precise querying and integration of factual data.

3. **Knowledge Graphs and Ontologies:**  
   - Graph databases representing enti

We see four messages (two interactions) because we used `k=4`.

On the plus side, we are shortening our conversation length when compared to buffer memory _without_ a window:

In [33]:
# Get window memory content
window_content = "\n".join([msg.content for msg in bufw_history])

print(
    f'Buffer memory conversation length: {len(tokenizer.encode(buffer_content))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(summary_content))}\n'
    f'Buffer window memory conversation length: {len(tokenizer.encode(window_content))}'
)

Buffer memory conversation length: 1274
Summary memory conversation length: 139
Buffer window memory conversation length: 705


_Practical Note: We are using `k=4` here for illustrative purposes, in most real world applications you would need a higher value for k._

### More memory types!

Given that we understand memory already, we will present a few more memory types here and hopefully a brief description will be enough to understand their underlying functionality.

#### Windows + Summary Hybrid

The following is a modern LCEL-compatible alternative to `ConversationSummaryBufferMemory`.

**Key feature:** _the conversation summary buffer memory keeps a summary of the earliest pieces of conversation while retaining a raw recollection of the latest interactions._

This combines the benefits of both summary and buffer window memory. Let's implement it:

In [34]:
class ConversationSummaryBufferMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)
    k: int = Field(default_factory=int)

    def __init__(self, llm: ChatOpenAI, k: int):
        super().__init__(llm=llm, k=k)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages and summarizing the messages that we drop.
        """
        existing_summary = None
        old_messages = None

        # See if we already have a summary message
        if len(self.messages) > 0 and isinstance(self.messages[0], SystemMessage):
            existing_summary = self.messages.pop(0)

        # Add the new messages to the history
        self.messages.extend(messages)

        # Check if we have too many messages
        if len(self.messages) > self.k:
            # Pull out the oldest messages...
            old_messages = self.messages[:-self.k]
            # ...and keep only the most recent messages
            self.messages = self.messages[-self.k:]

        if old_messages is None:
            # If we have no old_messages, we have nothing to update in summary
            return

        # Construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensure to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{old_messages}"
            )
        ])

        # Format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=existing_summary or "No previous summary",
                old_messages=old_messages
            )
        )

        # Prepend the new summary to the history
        self.messages = [SystemMessage(content=new_summary.content)] + self.messages

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

## What else can we do with memory?

There are several cool things we can do with memory in langchain:
* Implement our own custom memory modules (as we've done above)
* Use multiple memory modules in the same chain
* Combine agents with memory and other tools
* Integrate knowledge graphs

